In [3]:
import numpy as np
import tensorflow as tf
import pandas as pd
import os

# TODO: REMOVE 0 AND 1 FROM TRANSFORMER!! Current best acc: 94.3
labels = {
    0: '_silence_',
    1: 'unknown',
    2: 'bird',
    3: 'eight',
    4: 'falcon',
    5: 'five',
    6: 'four',
    7: 'nine',
    8: 'one',
    9: 'seven',
    10: 'six',
    11: 'snake',
    12: 'three',
    13: 'two',
    14: 'zero',
    }

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="models_data_v2_12_labels/96.456/tflite_non_stream/non_stream.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

audio, _ = tf.audio.decode_wav(tf.io.read_file('s1_test_release/s1_test_0003.wav'))  # snake
waveform = tf.expand_dims(tf.squeeze(audio, axis=-1), axis=-2)
# Test the model on random input data.
input_shape = input_details[0]['shape']
input_data = waveform
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
softmax_output = tf.nn.softmax(output_data)[0].numpy()
print(labels[np.argmax(softmax_output)])

predicted_labels, filenames = [], []
test_files = tf.io.gfile.glob('s1_test_release_normalized/*')
print('Test set size', len(test_files))
i = 0
print('Predicting...')
for audio in test_files:
    audio, _ = tf.audio.decode_wav(tf.io.read_file(audio))
    waveform = tf.expand_dims(tf.squeeze(audio, axis=-1), axis=-2)
    filenames.append(os.path.basename(test_files[i].replace('_normalized', '')))
    interpreter.set_tensor(input_details[0]['index'], waveform)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    softmax_output = tf.nn.softmax(output_data)[0].numpy()
    predicted_labels.append(labels[np.argmax(softmax_output)])
    i+=1

df = pd.DataFrame(list(zip(filenames, predicted_labels)))
df.to_csv('challenge_2_team_Tensor is not flowing.csv', index=False, header=False)  # tested submission file format passed
print(df)


snake
Test set size 649
Predicting...
                    0       1
0    s1_test_0000.wav   seven
1    s1_test_0001.wav  falcon
2    s1_test_0002.wav    four
3    s1_test_0003.wav   snake
4    s1_test_0004.wav    four
..                ...     ...
644  s1_test_0644.wav   seven
645  s1_test_0645.wav     one
646  s1_test_0646.wav    five
647  s1_test_0647.wav  falcon
648  s1_test_0648.wav     two

[649 rows x 2 columns]
